In [1]:
import pandas as pd
import turicreate as tc
import mysql.connector as db
from sklearn.model_selection import train_test_split

In [56]:
mydb = db.connect(
  host = "localhost",
  user = "root",
  passwd = "",
  database = "ecomm_ml"
)

lineItems = mydb.cursor()
lineItems.execute("SELECT `customer_id`, `product_id`, `quantity` FROM line_items")
lineItems = lineItems.fetchall()

customers = mydb.cursor()
customers.execute("SELECT `id` from `customers`")
customers = customers.fetchall()

In [57]:
lineItems = pd.DataFrame(lineItems,
                         columns=['customer_id', 'product_id', 'quantity'])

In [58]:
customers_list = []
for i in customers:
    customers_list.append(i[0])

In [59]:
def const_quantity(data):
    '''
    The constant data had to be created
    to be able to run RMSE tests
    :param data:
    :return data:
    '''
    data['quantity_one'] = 1
    return data

In [60]:
lineItems = lineItems.groupby(['customer_id', 'product_id'],
                              as_index=False)['quantity'].sum()

In [61]:
lineItemsConst = const_quantity(lineItems)

In [62]:
lineItemsFrame = tc.SFrame(lineItems)
lineItemsConstFrame = tc.SFrame(lineItemsConst)

In [63]:
popularity_model = tc.popularity_recommender.create(lineItemsFrame,
                                                    user_id="customer_id",
                                                    item_id="product_id",
                                                    verbose=False)
popularity_recommendations = popularity_model.recommend(users=customers_list,
                                                        k=3,
                                                        verbose=False)
popularity_recommendations.print_rows(20)

+-------------+---------------+--------+------+
| customer_id |   product_id  | score  | rank |
+-------------+---------------+--------+------+
|      1      | 1466095861859 | 1122.0 |  1   |
|      1      | 1466095894627 | 944.0  |  2   |
|      1      | 1466096386147 | 603.0  |  3   |
|      2      | 1466096189539 | 4427.0 |  1   |
|      2      | 1466095829091 | 1824.0 |  2   |
|      2      | 1466095861859 | 1122.0 |  3   |
|      3      | 1466096189539 | 4427.0 |  1   |
|      3      | 1466095960163 | 2002.0 |  2   |
|      3      | 1466095861859 | 1122.0 |  3   |
|      4      | 1466096189539 | 4427.0 |  1   |
|      4      | 1466095960163 | 2002.0 |  2   |
|      4      | 1466095829091 | 1824.0 |  3   |
|      5      | 1466096189539 | 4427.0 |  1   |
|      5      | 1466095960163 | 2002.0 |  2   |
|      5      | 1466095829091 | 1824.0 |  3   |
|      6      | 1466096189539 | 4427.0 |  1   |
|      6      | 1466095829091 | 1824.0 |  2   |
|      6      | 1466095861859 | 1122.0 |

In [64]:
similarity_model = tc.item_similarity_recommender.create(lineItemsConstFrame,
                                                         user_id="customer_id",
                                                         item_id="product_id",
                                                         target="quantity_one",
                                                         similarity_type='cosine',
                                                         verbose=False)
similarity_recommendations = similarity_model.recommend(users=customers_list,
                                                        k=3,
                                                        verbose=False)
similarity_recommendations.print_rows(20)

+-------------+---------------+----------------------+------+
| customer_id |   product_id  |        score         | rank |
+-------------+---------------+----------------------+------+
|      1      | 1466095861859 |  0.2105178435643514  |  1   |
|      1      | 1466095894627 |  0.0936524470647176  |  2   |
|      1      | 1466095927395 | 0.07460953791936238  |  3   |
|      2      | 1466096189539 |  0.2571062743663788  |  1   |
|      2      | 1466095829091 | 0.16636976599693298  |  2   |
|      2      | 1466095861859 | 0.14751386642456055  |  3   |
|      3      | 1466096189539 | 0.34698420763015747  |  1   |
|      3      | 1466095960163 | 0.24281388521194458  |  2   |
|      3      | 1466095861859 |  0.1859399676322937  |  3   |
|      4      | 1466096189539 | 0.10062533617019653  |  1   |
|      4      | 1466095829091 | 0.06592704568590436  |  2   |
|      4      | 1466095960163 | 0.06365778616496495  |  3   |
|      5      | 1239867981830 | 0.024802684783935547 |  1   |
|      5

In [65]:
delcursor = mydb.cursor()
sql = "DELETE FROM popularity"
delcursor.execute(sql)

for pop_rec in popularity_recommendations:
    
    mycursor = mydb.cursor()
    
    sql = "INSERT INTO popularity (customer_id, product_id, score, rank) VALUES (%s, %s, %s, %s)"
    val = (pop_rec['customer_id'], pop_rec['product_id'], pop_rec['score'], pop_rec['rank'])
    
    mycursor.execute(sql, val)
    mydb.commit()

In [66]:
delcursor = mydb.cursor()
sql = "DELETE FROM similarities"
delcursor.execute(sql)

for sim_rec in similarity_recommendations:
    
    mycursor = mydb.cursor()
    
    sql = "INSERT INTO similarities (customer_id, product_id, score, rank) VALUES (%s, %s, %s, %s)"
    val = (sim_rec['customer_id'], sim_rec['product_id'], sim_rec['score'], sim_rec['rank'])
    
    mycursor.execute(sql, val)
    mydb.commit()